In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from prince import MCA  # Para Análisis de Correspondencias Múltiples (MCA)

# Título de la aplicación
st.title('ReDim')

# Mensaje de bienvenida
st.write('Bienvenido a ReDim, una herramienta interactiva para realizar análisis exploratorio de datos (EDA), PCA y MCA.')

# Cargar archivo CSV
uploaded_file = st.file_uploader("Sube tu archivo CSV", type="csv")
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("Primeras filas del archivo:")
    st.write(df.head())

    # Estadísticas descriptivas
    st.write("Resumen descriptivo:")
    st.write(df.describe())

    # Valores nulos
    st.write("Valores nulos por columna:")
    st.write(df.isnull().sum())

    # Columnas numéricas
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

    # Botón de EDA
    if st.button('Ejecutar EDA'):
        st.write("Boxplots de columnas numéricas:")
        for col in numeric_cols:
            plt.figure(figsize=(8, 4))
            sns.boxplot(x=df[col])
            plt.title(f'Boxplot de {col}')
            st.pyplot(plt)

        st.write("Matriz de correlación:")
        corr_matrix = df[numeric_cols].corr()
        plt.figure(figsize=(10, 6))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        st.pyplot(plt)

    # Botón de PCA
    if st.button('Ejecutar PCA'):
        if len(numeric_cols) > 0:
            # Estandarizar los datos
            scaler = StandardScaler()
            df_scaled = scaler.fit_transform(df[numeric_cols])

            # PCA
            pca = PCA()
            pca_result = pca.fit_transform(df_scaled)

            # Matriz de covarianza
            st.write("Matriz de Covarianza:")
            st.write(np.cov(df_scaled.T))

            # Valores propios (autovalores)
            st.write("Valores propios (autovalores) de los componentes principales:")
            st.write(pca.explained_variance_)

            # Varianza explicada
            st.write("Varianza explicada por cada componente:")
            st.bar_chart(pca.explained_variance_ratio_)

            # Diagrama de dispersión
            plt.figure(figsize=(10, 6))
            plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.5)
            plt.title('PCA: PC1 vs PC2')
            plt.xlabel('PC1')
            plt.ylabel('PC2')
            st.pyplot(plt)

    # Botón de MCA
    if st.button('Ejecutar MCA'):
        # Seleccionar columnas categóricas
        cat_cols = df.select_dtypes(include=['object']).columns
        if len(cat_cols) > 1:  # Verificar que haya suficientes columnas categóricas
            mca = MCA(n_components=2, random_state=42)
            df_mca = df[cat_cols].dropna()
            mca = mca.fit(df_mca)

            # Inercia explicada
            eigenvalues = mca.eigenvalues_
            total_inertia = sum(eigenvalues)
            explained_inertia = [eig / total_inertia for eig in eigenvalues]

            # Mostrar la inercia explicada
            st.write("Proporción de inercia explicada por cada dimensión:")
            st.bar_chart(explained_inertia)

            # Transformar datos al espacio MCA
            mca_coordinates = mca.transform(df_mca)

            # Gráfico MCA
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.scatter(mca_coordinates.iloc[:, 0], mca_coordinates.iloc[:, 1], alpha=0.6, label="Observaciones", color='blue')

            # Coordenadas de las categorías
            category_coords = mca.column_coordinates(df_mca)
            for i, category in enumerate(category_coords.index):
                x, y = category_coords.iloc[i, :2]
                ax.arrow(0, 0, x, y, color='red', alpha=0.7, width=0.01, head_width=0.05)
                ax.text(x * 1.15, y * 1.15, category, color='darkred', fontsize=10, ha='center', va='center')

            ax.axhline(0, linestyle='--', color='grey', linewidth=0.5)
            ax.axvline(0, linestyle='--', color='grey', linewidth=0.5)
            ax.set_title("MCA: Dim1 vs Dim2")
            ax.set_xlabel("Dimensión 1")
            ax.set_ylabel("Dimensión 2")
            ax.legend()
            st.pyplot(fig)

        else:
            st.write("No hay suficientes columnas categóricas para realizar MCA.")

    st.write("¡Gracias por usar ReDim!")


ModuleNotFoundError: No module named 'streamlit'